In [23]:
# The aim of this script is to download all rental property data and all available property from realestate website.
# the looping method doesn't work, hence the long winded version

# pip install -e git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api

from realestate_com_au import RealestateComAu
api = RealestateComAu()

# Get property listings
# listings = api.search(locations=["seventeen seventy, qld 4677"], channel="buy", keywords=["tenant"], exclude_keywords=["pool"])

import pandas as pd

In [29]:

# RAW_RENT = api.search(locations=['2127,Wentworth Point'],channel="rent")
# RAW_RENT_DF = pd.DataFrame(RAW_RENT)
# RAW_RENT1 = RAW_RENT_DF.drop(columns=['id','images','images_floorplans','listers'])
# RAW_RENT_UNIQ = RAW_RENT1.drop_duplicates()
print("finished rent extract")

# Everything within the postcode
RAW_ALL = api.search(locations=['2127,Wentworth Point'] )
RAW_ALL_DF = pd.DataFrame(RAW_ALL)
RAW_ALL1 = RAW_ALL_DF.drop(columns=['id','images','images_floorplans','listers'])
RAW_ALL_UNIQ = RAW_ALL1.drop_duplicates()
print("finished all extract")

# del RAW_RENT, RAW_RENT_DF, RAW_RENT1, RAW_ALL, RAW_ALL_DF, RAW_ALL1

finished rent extract


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [25]:
# consolidate for aggregation comparisons

INFO_ALL = RAW_ALL_UNIQ.reset_index() 
INFO_RENT = RAW_RENT_UNIQ.reset_index()


# REF_PCODE_SBRB = REF_PCODE_SBRB[["postcode","suburb"]]
# full comparison version
REF_PCODE_SBRB = INFO_ALL[["postcode","suburb","property_type","bedrooms","bathrooms"]]
REF_PCODE_SBRB = REF_PCODE_SBRB.drop_duplicates()


# Remove rentals from the masterlist to create a list of properties to buy
#   Note: maybe need to add a few extra filters to find the buy only population
A_FIND_BUY = INFO_ALL.merge(INFO_RENT,how='outer',on=["postcode","suburb","property_type","bedrooms","bathrooms"], indicator= True)
INFO_BUY = A_FIND_BUY[A_FIND_BUY['_merge']=="left_only"]
INFO_BUY = INFO_BUY.reset_index()
INFO_BUY.columns = INFO_BUY.columns.str.replace("_x","")
del A_FIND_BUY


In [26]:
# Rental data
INFO_RENT_1 = INFO_RENT[['suburb','state','postcode','short_address','full_address','property_type','price','price_text','bedrooms','bathrooms','parking_spaces','building_size','building_size_unit','land_size','land_size_unit']].fillna(0)

# INFO_RENT_1 = R_INFO_ALL.drop(columns=['badge','url','listing_company_name','listing_company_phone','description']).fillna(0)
INFO_RENT_1 = INFO_RENT_1.drop_duplicates()
# summarise by postcode and suburb

# INFO_RENT_MED = INFO_RENT_1.groupby(["postcode","suburb"])["price"].median().reset_index()
INFO_RENT_MED = INFO_RENT_1.groupby(["postcode","suburb","property_type","bedrooms"])["price"].median().reset_index()
INFO_RENT_MED.rename(columns={'price':'price_RENT'}, inplace = True)

# INFO_RENT_FREQ = INFO_RENT_1.groupby(["postcode","suburb"])["full_address"].count().reset_index()
INFO_RENT_FREQ = INFO_RENT_1.groupby(["postcode","suburb","property_type","bedrooms"])["full_address"].count().reset_index()
INFO_RENT_FREQ.rename(columns={'full_address':'num_RENT'}, inplace = True)

# del INFO_RENT_1 

In [27]:
# Sales data
INFO_BUY_1 = INFO_BUY[['suburb','state','postcode','short_address','full_address','property_type','price','price_text','bedrooms','bathrooms','parking_spaces','building_size','building_size_unit','land_size','land_size_unit']].fillna(0)
# A_INFO_SUBSET= INFO_BUY_.reset_index()
INFO_BUY_1.drop_duplicates()
# summarise by postcode and suburb

# INFO_BUY_MED = INFO_BUY_1.groupby(["postcode","suburb"])["price"].median().reset_index()
INFO_BUY_MED = INFO_BUY_1.groupby(["postcode","suburb","property_type","bedrooms"])["price"].median().reset_index()
INFO_BUY_MED.rename(columns={'price':'price_BUY'}, inplace = True)

# INFO_BUY_FREQ = INFO_BUY_1.groupby(["postcode","suburb"])["full_address"].count().reset_index()
INFO_BUY_FREQ = INFO_BUY_1.groupby(["postcode","suburb","property_type","bedrooms"])["full_address"].count().reset_index()
INFO_BUY_FREQ.rename(columns={'full_address':'num_BUY'}, inplace = True)

# del INFO_BUY_1

In [28]:
df_comp = REF_PCODE_SBRB.merge(INFO_BUY_MED,how='left',on=["postcode","suburb","property_type","bedrooms"]).merge(INFO_RENT_MED,how='left',on=["postcode","suburb","property_type","bedrooms"]).fillna(0)
df_comp = df_comp.merge(INFO_BUY_FREQ,how='left',on=["postcode","suburb","property_type","bedrooms"]).merge(INFO_RENT_FREQ,how='left',on=["postcode","suburb","property_type","bedrooms"]).fillna(0)

df_comp.dtypes
# df_comp = REF_PCODE_SBRB.merge(R_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(R_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)
# df_comp = df_comp.merge(A_INFO_SUBSET_MED,how='left',on=["postcode","suburb"]).merge(A_INFO_SUBSET_FREQ,how='left',on=["postcode","suburb"]).fillna(0)

# if pd.to_numeric(df_comp['price_RENT']) == 0:
    # df_comp['YEARLY_RENT']  = 0
    # df_comp['RENT_YIELD']   = 0
    # df_comp['RENT_DIV_BUY'] = 0
# else:
df_comp['YEARLY_RENT']=df_comp['price_RENT']*52
df_comp['RENT_YIELD']=df_comp['YEARLY_RENT']/df_comp['price_BUY']
df_comp['RENT_DIV_BUY'] = df_comp['num_RENT'] / df_comp['num_BUY']



        # if INF['dividendRate'] is not None:
        #     INF['dr']=INF['dividendRate']
        # else:
        #     INF['dr']=0


# create combo key
# df_comp['Property Desc'] = df_comp['postcode']+' ('+df_comp['suburb']+') :'+df_comp['property_type']+' '+df_comp['property_type']+str(df_comp['bedrooms'])+'bedroom '+str(df_comp['bathrooms'])+'baths'

# del R_INFO_SUBSET_MED, R_INFO_SUBSET_FREQ, A_INFO_SUBSET_MED, A_INFO_SUBSET_FREQ
# 7303